In [ ]:
# Data Augmentation and L2 Regularization with Dropout and Early stopping

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"isaac000o","key":"2c1a236c98c03c0ce703259671092452"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d legendahmed/alzheimermridataset

 79% 49.0M/61.8M [00:00<00:00, 226MB/s]
100% 61.8M/61.8M [00:00<00:00, 244MB/s]


In [ ]:
from zipfile import ZipFile
filename = "alzheimermridataset.zip"

with ZipFile(filename,'r') as zip:
  print("Extracting of the file ...")
  zip.extractall()
  print("Done")

Extracting of the file ...
Done


In [ ]:
#libraries 
from imutils import paths
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Dropout, Flatten, Dense, MaxPool2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from matplotlib import pyplot
import tensorflow as tf
from keras.models import load_model



In [ ]:
Training_images_paths = list(paths.list_images("/content/Alzheimer_s Dataset/train"))
Training_labels = []
Training_images = []

i = 0 
while i < len(Training_images_paths):
  Training_labels.append(Training_images_paths[i].split("/")[4])
  image = cv2.imread(Training_images_paths[i])
  image = cv2.resize(image,(256,256))
  Training_images.append(image)
  i = i + 1



print("Number of Images:",len(Training_labels))



Testing_images_paths = list(paths.list_images("/content/Alzheimer_s Dataset/test"))
Testing_labels = []
Testing_images = []

i = 0 
while i < len(Testing_images_paths):
  Testing_labels.append(Testing_images_paths[i].split("/")[4])
  image = cv2.imread(Testing_images_paths[i])
  image = cv2.resize(image,(256,256))
  Testing_images.append(image)
  i = i + 1



print("Number of Images:",len(Testing_labels))


Number of Images: 5121
Number of Images: 1279


In [ ]:
data_generator = ImageDataGenerator(horizontal_flip=True)

In [ ]:
Training_images_np = np.array(Training_images) 
Training_labels_np = np.array(Training_labels)

Testing_images_np = np.array(Testing_images) 
Testing_labels_np = np.array(Testing_labels)



In [ ]:
aug_traning_images_VeryMildDemented = []
aug_traning_labels_VeryMildDemented = []
v = 0 
while v < 1792:
  traning_image = Training_images_np[v]
  traning_image = traning_image.reshape((1,)+traning_image.shape)
  it = data_generator.flow(traning_image,batch_size=1,shuffle=True)

  for i in range(10):
    batch = it.next()
    aug_traning_image = batch[0].astype('uint8') 
    aug_traning_images_VeryMildDemented.append(aug_traning_image)
    aug_traning_labels_VeryMildDemented.append(Training_labels[v])
  v = v+1

print(len(aug_traning_labels_VeryMildDemented))

17920


In [ ]:
aug_traning_images_ModerateDemented = []
aug_traning_labels_ModerateDemented = []

v = 1792 
while v < 1844:
  traning_image = Training_images_np[v]
  traning_image = traning_image.reshape((1,)+traning_image.shape)
  it = data_generator.flow(traning_image,batch_size=1,shuffle=True)

  for i in range(400):
    batch = it.next()
    aug_traning_image = batch[0].astype('uint8') 
    aug_traning_images_ModerateDemented.append(aug_traning_image)
    aug_traning_labels_ModerateDemented.append(Training_labels[v])
  v = v+1
print(len(aug_traning_labels_ModerateDemented))

20800


In [ ]:
aug_traning_images_NonDemented = []
aug_traning_labels_NonDemented = []

v = 1844 
while v < 4404:
  traning_image = Training_images_np[v]
  traning_image = traning_image.reshape((1,)+traning_image.shape)
  it = data_generator.flow(traning_image,batch_size=1,shuffle=True)

  for i in range(6):
    batch = it.next()
    aug_traning_image = batch[0].astype('uint8') 
    aug_traning_images_NonDemented.append(aug_traning_image)
    aug_traning_labels_NonDemented.append(Training_labels[v])
  v = v+1



print(len(aug_traning_labels_NonDemented))

15360


In [ ]:
aug_traning_images_MildDemented = []
aug_traning_labels_MildDemented = []

v = 4404 
while v < len(Training_labels):
  traning_image = Training_images_np[v]
  traning_image = traning_image.reshape((1,)+traning_image.shape)
  it = data_generator.flow(traning_image,batch_size=1,shuffle=True)

  for i in range(25):
    batch = it.next()
    aug_traning_image = batch[0].astype('uint8') 
    aug_traning_images_MildDemented.append(aug_traning_image)
    aug_traning_labels_MildDemented.append(Training_labels[v])
  v = v+1


print(len(aug_traning_labels_MildDemented))

17925


In [ ]:
aug_traning_images = aug_traning_images_MildDemented + aug_traning_images_NonDemented + aug_traning_images_ModerateDemented + aug_traning_images_VeryMildDemented+Training_images
aug_traning_labels = aug_traning_labels_MildDemented + aug_traning_labels_NonDemented + aug_traning_labels_ModerateDemented + aug_traning_labels_VeryMildDemented+Training_labels

print(len(aug_traning_images))
print(len(aug_traning_labels))


77126
77126


In [ ]:
aug_testing_images_VeryMildDemented = []
aug_testing_labels_VeryMildDemented = []
x = 0 
while x < 448:
  testing_image = Testing_images_np[x]
  testing_image = testing_image.reshape((1,)+testing_image.shape)
  it = data_generator.flow(testing_image,batch_size=1,shuffle=True)

  for i in range(9):
    batch = it.next()
    aug_testing_image = batch[0].astype('uint8') 
    aug_testing_images_VeryMildDemented.append(aug_testing_image)
    aug_testing_labels_VeryMildDemented.append(Testing_labels[x])
  x = x+1

print(len(aug_testing_labels_VeryMildDemented))

4032


In [ ]:
aug_testing_images_ModerateDemented = []
aug_testing_labels_ModerateDemented = []
x = 448 
while x < 460:
  testing_image = Testing_images_np[x]
  testing_image = testing_image.reshape((1,)+testing_image.shape)
  it = data_generator.flow(testing_image,batch_size=1,shuffle=True)

  for i in range(400):
    batch = it.next()
    aug_testing_image = batch[0].astype('uint8') 
    aug_testing_images_ModerateDemented.append(aug_testing_image)
    aug_testing_labels_ModerateDemented.append(Testing_labels[x])
  x = x+1

print(len(aug_testing_labels_ModerateDemented))

4800


In [ ]:
aug_testing_images_NonDemented = []
aug_testing_labels_NonDemented = []
x = 460 
while x < 1100:
  testing_image = Testing_images_np[x]
  testing_image = testing_image.reshape((1,)+testing_image.shape)
  it = data_generator.flow(testing_image,batch_size=1,shuffle=True)

  for i in range(8):
    batch = it.next()
    aug_testing_image = batch[0].astype('uint8') 
    aug_testing_images_NonDemented.append(aug_testing_image)
    aug_testing_labels_NonDemented.append(Testing_labels[x])
  x = x+1

print(len(aug_testing_labels_NonDemented))


5120


In [ ]:
aug_testing_images_MildDemented = []
aug_testing_labels_MildDemented = []
x = 1100 

while x < len(Testing_labels):
  testing_image = Testing_images_np[x]
  testing_image = testing_image.reshape((1,)+testing_image.shape)
  it = data_generator.flow(testing_image,batch_size=1,shuffle=True)
  #130
  for i in range(30):
    batch = it.next()
    aug_testing_image = batch[0].astype('uint8') 
    aug_testing_images_MildDemented.append(aug_testing_image)
    aug_testing_labels_MildDemented.append(Testing_labels[x])
  x = x+1

print(len(aug_testing_labels_MildDemented))

5370


In [ ]:
aug_testing_images = aug_testing_images_MildDemented + aug_testing_images_NonDemented + aug_testing_images_ModerateDemented + aug_testing_images_VeryMildDemented + Testing_images
aug_testing_labels = aug_testing_labels_MildDemented + aug_testing_labels_NonDemented + aug_testing_labels_ModerateDemented + aug_testing_labels_VeryMildDemented + Testing_labels

print(len(aug_testing_images))
print(len(aug_testing_labels))

20601
20601


In [ ]:
i = 0 
while i <len(aug_traning_images):
  image = aug_traning_images[i]
  image = cv2.resize(image,(128,128))
  image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
  aug_traning_images[i] = image
  i = i + 1


n = 0 
while n < len(aug_testing_images):
  image = aug_testing_images[n]
  image = cv2.resize(image,(128,128))
  image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
  aug_testing_images[n] = image
  n = n + 1

print(type(aug_traning_images))
aug_traning_images = np.array(aug_traning_images)
aug_testing_images = np.array(aug_testing_images)
print(type(aug_traning_images))


aug_traning_images = aug_traning_images/255.00
aug_testing_images = aug_testing_images/255.00

label_encoder = LabelEncoder()

aug_traning_labels = label_encoder.fit_transform(aug_traning_labels)
print(aug_traning_labels.shape)

label_encoder = LabelEncoder()
aug_testing_labels = label_encoder.fit_transform(aug_testing_labels)
print(aug_testing_labels.shape)

aug_traning_labels = to_categorical(aug_traning_labels)
aug_testing_labels = to_categorical(aug_testing_labels)

aug_traning_images = np.expand_dims(aug_traning_images, axis=-1)
aug_testing_images = np.expand_dims(aug_testing_images, axis=-1)


print(aug_traning_images.shape)
print(aug_traning_labels.shape)
print(aug_testing_images.shape)
print(aug_testing_labels.shape)

<class 'list'>
<class 'numpy.ndarray'>
(77126,)
(20601,)
(77126, 128, 128, 1)
(77126, 4)
(20601, 128, 128, 1)
(20601, 4)


In [1]:
def Alzheimer_Data_Augmentation_L2_Dropout_Earlystopping(input_shape=(128, 128, 1)):
    regularizer = tf.keras.regularizers.l2(0.001)
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Conv2D(128, (5, 5), padding='same', strides=(1, 1), name='conv1', activation='sigmoid', kernel_regularizer=regularizer))
    model.add(MaxPool2D((9, 9), strides=(3, 3)))
    model.add(Conv2D(64, (5, 5), padding='same', strides=(1, 1), name='conv2', activation='sigmoid', kernel_regularizer=regularizer))
    model.add(MaxPool2D((7, 7), strides=(3, 3)))
    model.add(Conv2D(32, (3, 3), padding='same', strides=(1, 1), name='conv3', activation='sigmoid', kernel_regularizer=regularizer))
    model.add(MaxPool2D((5, 5), strides=(2, 2)))
 #   model.add(Conv2D(32, (3, 3), padding='same', strides=(1, 1), name='conv4', activation='relu', kernel_regularizer=regularizer))
 #   model.add(MaxPool2D((3, 3), strides=(2, 2)))    
    
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='sigmoid'))
    model.add(Dropout(0.5))
    model.add(Dense(4, activation='sigmoid'))
    optimizer = Adam(3.15e-5)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [2]:
model= Alzheimer_Data_Augmentation_L2_Dropout_Earlystopping(input_shape=(128, 128, 1))
model.summary()

NameError: ignored

In [ ]:
#Setting up Early Stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

In [ ]:
hist_ = model.fit(aug_traning_images, aug_traning_labels, batch_size=128, epochs=200, validation_data=(aug_testing_images, aug_testing_labels), callbacks=[es, mc])
saved_model = load_model('best_model.h5')
# evaluate the model
_, train_acc = saved_model.evaluate(aug_testing_images, aug_testing_labels, verbose=0)
_, test_acc = saved_model.evaluate(aug_testing_images, aug_testing_labels, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

Epoch 1/200
603/603 [==============================] - ETA: 0s - loss: 1.2663 - accuracy: 0.5182
Epoch 00001: val_accuracy improved from -inf to 0.52968, saving model to best_model.h5
603/603 [==============================] - 58s 96ms/step - loss: 1.2663 - accuracy: 0.5182 - val_loss: 1.1628 - val_accuracy: 0.5297
Epoch 2/200
602/603 [============================>.] - ETA: 0s - loss: 1.1341 - accuracy: 0.5602
Epoch 00002: val_accuracy did not improve from 0.52968
603/603 [==============================] - 55s 91ms/step - loss: 1.1341 - accuracy: 0.5601 - val_loss: 1.1227 - val_accuracy: 0.5297
Epoch 3/200
602/603 [============================>.] - ETA: 0s - loss: 1.0904 - accuracy: 0.5761
Epoch 00003: val_accuracy did not improve from 0.52968
603/603 [==============================] - 55s 91ms/step - loss: 1.0904 - accuracy: 0.5761 - val_loss: 1.0975 - val_accuracy: 0.5297
Epoch 4/200
602/603 [============================>.] - ETA: 0s - loss: 1.0640 - accuracy: 0.5831
Epoch 00004: val

In [ ]:
def Alzheimer_Data_Augmentation_L2_Dropout_Earlystopping(input_shape=(128, 128, 1)):
    regularizer = tf.keras.regularizers.l2(0.001)
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Conv2D(128, (5, 5), padding='same', strides=(1, 1), name='conv1', activation='sigmoid', kernel_regularizer=regularizer))
    model.add(MaxPool2D((9, 9), strides=(3, 3)))
    model.add(Conv2D(64, (5, 5), padding='same', strides=(1, 1), name='conv2', activation='sigmoid', kernel_regularizer=regularizer))
    model.add(MaxPool2D((7, 7), strides=(3, 3)))
    model.add(Conv2D(32, (3, 3), padding='same', strides=(1, 1), name='conv3', activation='sigmoid', kernel_regularizer=regularizer))
    model.add(MaxPool2D((5, 5), strides=(2, 2)))
    model.add(Conv2D(32, (3, 3), padding='same', strides=(1, 1), name='conv4', activation='sigmoid', kernel_regularizer=regularizer))
    model.add(MaxPool2D((3, 3), strides=(2, 2)))    
    
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='sigmoid'))
    model.add(Dropout(0.5))
    model.add(Dense(4, activation='softmax'))
    optimizer = Adam(3.15e-5)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model= Alzheimer_Data_Augmentation_L2_Dropout_Earlystopping(input_shape=(128, 128, 1))


  #Setting up Early Stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

hist_ = model.fit(aug_traning_images, aug_traning_labels, batch_size=128, epochs=200, validation_data=(aug_testing_images, aug_testing_labels), callbacks=[es, mc])
saved_model = load_model('best_model.h5')
# evaluate the model
_, train_acc = saved_model.evaluate(aug_testing_images, aug_testing_labels, verbose=0)
_, test_acc = saved_model.evaluate(aug_testing_images, aug_testing_labels, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))


Epoch 1/200
603/603 [==============================] - ETA: 0s - loss: 1.6234 - accuracy: 0.3343
Epoch 00001: val_accuracy improved from -inf to 0.52968, saving model to best_model.h5
603/603 [==============================] - 65s 94ms/step - loss: 1.6234 - accuracy: 0.3343 - val_loss: 1.2204 - val_accuracy: 0.5297
Epoch 2/200
602/603 [============================>.] - ETA: 0s - loss: 1.2632 - accuracy: 0.4929
Epoch 00002: val_accuracy did not improve from 0.52968
603/603 [==============================] - 55s 91ms/step - loss: 1.2632 - accuracy: 0.4929 - val_loss: 1.1403 - val_accuracy: 0.5297
Epoch 3/200
602/603 [============================>.] - ETA: 0s - loss: 1.1877 - accuracy: 0.5185
Epoch 00003: val_accuracy did not improve from 0.52968
603/603 [==============================] - 55s 91ms/step - loss: 1.1877 - accuracy: 0.5185 - val_loss: 1.1081 - val_accuracy: 0.5297
Epoch 4/200
603/603 [==============================] - ETA: 0s - loss: 1.1440 - accuracy: 0.5248
Epoch 00004: val

In [ ]:
def Alzheimer_Data_Augmentation_L2_Dropout_Earlystopping(input_shape=(128, 128, 1)):
    regularizer = tf.keras.regularizers.l2(0.001)
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Conv2D(128, (5, 5), padding='same', strides=(1, 1), name='conv1', activation='tanh', kernel_regularizer=regularizer))
    model.add(MaxPool2D((9, 9), strides=(3, 3)))
    model.add(Conv2D(64, (5, 5), padding='same', strides=(1, 1), name='conv2', activation='tanh', kernel_regularizer=regularizer))
    model.add(MaxPool2D((7, 7), strides=(3, 3)))
    model.add(Conv2D(32, (3, 3), padding='same', strides=(1, 1), name='conv3', activation='tanh', kernel_regularizer=regularizer))
    model.add(MaxPool2D((5, 5), strides=(2, 2)))
    model.add(Conv2D(32, (3, 3), padding='same', strides=(1, 1), name='conv4', activation='tanh', kernel_regularizer=regularizer))
    model.add(MaxPool2D((3, 3), strides=(2, 2)))    
    
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(4, activation='softmax'))
    optimizer = Adam(3.15e-5)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model= Alzheimer_Data_Augmentation_L2_Dropout_Earlystopping(input_shape=(128, 128, 1))


  #Setting up Early Stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

hist_ = model.fit(aug_traning_images, aug_traning_labels, batch_size=128, epochs=200, validation_data=(aug_testing_images, aug_testing_labels), callbacks=[es, mc])
saved_model = load_model('best_model.h5')
# evaluate the model
_, train_acc = saved_model.evaluate(aug_testing_images, aug_testing_labels, verbose=0)
_, test_acc = saved_model.evaluate(aug_testing_images, aug_testing_labels, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))






InternalError: ignored

In [ ]:
def Alzheimer_Data_Augmentation_L2_Dropout_Earlystopping(input_shape=(128, 128, 1)):
    regularizer = tf.keras.regularizers.l2(0.001)
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Conv2D(128, (5, 5), padding='same', strides=(1, 1), name='conv1', activation='leaky_relu', kernel_regularizer=regularizer))
    model.add(MaxPool2D((9, 9), strides=(3, 3)))
    model.add(Conv2D(64, (5, 5), padding='same', strides=(1, 1), name='conv2', activation='leaky_relu', kernel_regularizer=regularizer))
    model.add(MaxPool2D((7, 7), strides=(3, 3)))
    model.add(Conv2D(32, (3, 3), padding='same', strides=(1, 1), name='conv3', activation='leaky_relu', kernel_regularizer=regularizer))
    model.add(MaxPool2D((5, 5), strides=(2, 2)))
    model.add(Conv2D(32, (3, 3), padding='same', strides=(1, 1), name='conv4', activation='leaky_relu', kernel_regularizer=regularizer))
    model.add(MaxPool2D((3, 3), strides=(2, 2)))    
    
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='leaky_relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4, activation='softmax'))
    optimizer = Adam(3.15e-5)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model= Alzheimer_Data_Augmentation_L2_Dropout_Earlystopping(input_shape=(128, 128, 1))


  #Setting up Early Stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

hist_ = model.fit(aug_traning_images, aug_traning_labels, batch_size=128, epochs=200, validation_data=(aug_testing_images, aug_testing_labels), callbacks=[es, mc])
saved_model = load_model('best_model.h5')
# evaluate the model
_, train_acc = saved_model.evaluate(aug_testing_images, aug_testing_labels, verbose=0)
_, test_acc = saved_model.evaluate(aug_testing_images, aug_testing_labels, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))


Epoch 1/200
603/603 [==============================] - ETA: 0s - loss: 1.3457 - accuracy: 0.4639
Epoch 00001: val_accuracy improved from -inf to 0.52968, saving model to best_model.h5
603/603 [==============================] - 68s 97ms/step - loss: 1.3457 - accuracy: 0.4639 - val_loss: 1.1877 - val_accuracy: 0.5297
Epoch 2/200
603/603 [==============================] - ETA: 0s - loss: 1.1427 - accuracy: 0.5696
Epoch 00002: val_accuracy did not improve from 0.52968
603/603 [==============================] - 57s 94ms/step - loss: 1.1427 - accuracy: 0.5696 - val_loss: 1.1110 - val_accuracy: 0.5297
Epoch 3/200
603/603 [==============================] - ETA: 0s - loss: 1.0477 - accuracy: 0.5932
Epoch 00003: val_accuracy improved from 0.52968 to 0.55643, saving model to best_model.h5
603/603 [==============================] - 57s 94ms/step - loss: 1.0477 - accuracy: 0.5932 - val_loss: 1.0412 - val_accuracy: 0.5564
Epoch 4/200
603/603 [==============================] - ETA: 0s - loss: 0.9865 

In [ ]:

def Alzheimer_Data_Augmentation_L2_Dropout_Earlystopping(input_shape=(128, 128, 1)):
    regularizer = tf.keras.regularizers.l2(0.001)
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Conv2D(128, (5, 5), padding='same', strides=(1, 1), name='conv1', activation='PReLU', kernel_regularizer=regularizer))
    model.add(MaxPool2D((9, 9), strides=(3, 3)))
    model.add(Conv2D(64, (5, 5), padding='same', strides=(1, 1), name='conv2', activation='PReLU', kernel_regularizer=regularizer))
    model.add(MaxPool2D((7, 7), strides=(3, 3)))
    model.add(Conv2D(32, (3, 3), padding='same', strides=(1, 1), name='conv3', activation='PReLU', kernel_regularizer=regularizer))
    model.add(MaxPool2D((5, 5), strides=(2, 2)))
    model.add(Conv2D(32, (3, 3), padding='same', strides=(1, 1), name='conv4', activation='PReLU', kernel_regularizer=regularizer))
    model.add(MaxPool2D((3, 3), strides=(2, 2)))    
    
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='PReLU'))
    model.add(Dropout(0.5))
    model.add(Dense(4, activation='softmax'))
    optimizer = Adam(3.15e-5)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model= Alzheimer_Data_Augmentation_L2_Dropout_Earlystopping(input_shape=(128, 128, 1))


  #Setting up Early Stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

hist_ = model.fit(aug_traning_images, aug_traning_labels, batch_size=128, epochs=200, validation_data=(aug_testing_images, aug_testing_labels), callbacks=[es, mc])
saved_model = load_model('best_model.h5')
# evaluate the model
_, train_acc = saved_model.evaluate(aug_testing_images, aug_testing_labels, verbose=0)
_, test_acc = saved_model.evaluate(aug_testing_images, aug_testing_labels, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))


Epoch 1/200
603/603 [==============================] - ETA: 0s - loss: 1.3032 - accuracy: 0.5208
Epoch 00001: val_accuracy improved from -inf to 0.52968, saving model to best_model.h5
603/603 [==============================] - 109s 179ms/step - loss: 1.3032 - accuracy: 0.5208 - val_loss: 1.1775 - val_accuracy: 0.5297
Epoch 2/200
603/603 [==============================] - ETA: 0s - loss: 1.1287 - accuracy: 0.5839
Epoch 00002: val_accuracy improved from 0.52968 to 0.53289, saving model to best_model.h5
603/603 [==============================] - 107s 177ms/step - loss: 1.1287 - accuracy: 0.5839 - val_loss: 1.0924 - val_accuracy: 0.5329
Epoch 3/200
603/603 [==============================] - ETA: 0s - loss: 1.0287 - accuracy: 0.6146
Epoch 00003: val_accuracy improved from 0.53289 to 0.58619, saving model to best_model.h5
603/603 [==============================] - 107s 177ms/step - loss: 1.0287 - accuracy: 0.6146 - val_loss: 1.0418 - val_accuracy: 0.5862
Epoch 4/200
603/603 [================

In [ ]:
def Alzheimer_Data_Augmentation_L2_Dropout_Earlystopping(input_shape=(128, 128, 1)):
    regularizer = tf.keras.regularizers.l2(0.001)
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Conv2D(128, (5, 5), padding='same', strides=(1, 1), name='conv1', activation='softmax', kernel_regularizer=regularizer))
    model.add(MaxPool2D((9, 9), strides=(3, 3)))
    model.add(Conv2D(64, (5, 5), padding='same', strides=(1, 1), name='conv2', activation='softmax', kernel_regularizer=regularizer))
    model.add(MaxPool2D((7, 7), strides=(3, 3)))
    model.add(Conv2D(32, (3, 3), padding='same', strides=(1, 1), name='conv3', activation='softmax', kernel_regularizer=regularizer))
    model.add(MaxPool2D((5, 5), strides=(2, 2)))
    model.add(Conv2D(32, (3, 3), padding='same', strides=(1, 1), name='conv4', activation='softmax', kernel_regularizer=regularizer))
    model.add(MaxPool2D((3, 3), strides=(2, 2)))    
    
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='softmax'))
    model.add(Dropout(0.5))
    model.add(Dense(4, activation='softmax'))
    optimizer = Adam(3.15e-5)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model= Alzheimer_Data_Augmentation_L2_Dropout_Earlystopping(input_shape=(128, 128, 1))


  #Setting up Early Stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

hist_ = model.fit(aug_traning_images, aug_traning_labels, batch_size=128, epochs=200, validation_data=(aug_testing_images, aug_testing_labels), callbacks=[es, mc])
saved_model = load_model('best_model.h5')
# evaluate the model
_, train_acc = saved_model.evaluate(aug_testing_images, aug_testing_labels, verbose=0)
_, test_acc = saved_model.evaluate(aug_testing_images, aug_testing_labels, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))


Epoch 1/200
603/603 [==============================] - ETA: 0s - loss: 1.4756 - accuracy: 0.4036
Epoch 00001: val_accuracy improved from -inf to 0.52968, saving model to best_model.h5
603/603 [==============================] - 81s 134ms/step - loss: 1.4756 - accuracy: 0.4036 - val_loss: 1.4232 - val_accuracy: 0.5297
Epoch 2/200
603/603 [==============================] - ETA: 0s - loss: 1.3866 - accuracy: 0.5886
Epoch 00002: val_accuracy did not improve from 0.52968
603/603 [==============================] - 78s 129ms/step - loss: 1.3866 - accuracy: 0.5886 - val_loss: 1.3684 - val_accuracy: 0.5297
Epoch 3/200
603/603 [==============================] - ETA: 0s - loss: 1.3415 - accuracy: 0.5909
Epoch 00003: val_accuracy did not improve from 0.52968
603/603 [==============================] - 78s 129ms/step - loss: 1.3415 - accuracy: 0.5909 - val_loss: 1.3360 - val_accuracy: 0.5297
Epoch 4/200
603/603 [==============================] - ETA: 0s - loss: 1.3103 - accuracy: 0.5909
Epoch 00004: 

KeyboardInterrupt: ignored